![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 4.Training and Reusing Text Classification Models

**Relevant blogpost:** https://towardsdatascience.com/text-classification-in-spark-nlp-with-bert-and-universal-sentence-encoders-e644d618ca32

In [0]:
import json
import os
import string
import pandas as pd
import numpy as np

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.util import *

import pyspark.sql.functions as F
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import functions as F


pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 100)

print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp_jsl.version : 3.0.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 spark://10.164.252.64:7077 
 AppName 
 Databricks Shell

## Using Pretrained ClassifierDL and SentimentDL models

In [0]:
fake_classifier = ClassifierDLModel.pretrained('classifierdl_use_fakenews', 'en') \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")

classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

fake_news classifier is trained on `https://raw.githubusercontent.com/joolsa/fake_real_news_dataset/master/fake_or_real_news.csv.zip`

In [0]:
fake_classifier.getClasses()

Out[3]: ['FAKE', 'REAL']

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(name="tfhub_use",lang="en") \
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      use,
      fake_classifier
  ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

fake_clf_model = nlpPipeline.fit(empty_data)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/spam_ham_dataset.csv
  
#dbutils.fs.cp("file:/databricks/driver/spam_ham_dataset.csv", "dbfs:/") 

In [0]:
fake_lp_pipeline = LightPipeline(fake_clf_model)

text = 'BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'


fake_lp_pipeline.annotate(text)

Out[6]: {'document': ['BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'],
 'sentence_embeddings': ['BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'],
 'class': ['FAKE']}

In [0]:
sample_data = spark.createDataFrame([[text]]).toDF("text")

sample_data.show(truncate=False)

+-------------------------------------------------------------------------------------------------+
text |
+-------------------------------------------------------------------------------------------------+
BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump|
+-------------------------------------------------------------------------------------------------+

In [0]:
pred = fake_clf_model.transform(sample_data)



In [0]:
pred.show()

+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence_embeddings| class|
+--------------------+--------------------+--------------------+--------------------+
BREAKING: Leaked ...|[{document, 0, 96...|[{sentence_embedd...|[{category, 0, 96...|
+--------------------+--------------------+--------------------+--------------------+

In [0]:
pred.select('text','class.result').show(truncate=False)

+-------------------------------------------------------------------------------------------------+------+
text |result|
+-------------------------------------------------------------------------------------------------+------+
BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump|[FAKE]|
+-------------------------------------------------------------------------------------------------+------+

you can find more samples here >> `https://github.com/KaiDMML/FakeNewsNet/tree/master/dataset`

## Generic classifier function

In [0]:
def get_clf_lp(model_name, sentiment_dl=False, pretrained=True):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  use = UniversalSentenceEncoder.pretrained(lang="en") \
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")


  if pretrained:

    if sentiment_dl:

      document_classifier = SentimentDLModel.pretrained(model_name, 'en') \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")
    else:
      document_classifier = ClassifierDLModel.pretrained(model_name, 'en') \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")

  else:

    if sentiment_dl:

      document_classifier = SentimentDLModel.load(model_name) \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")
    else:
      document_classifier = ClassifierDLModel.load(model_name) \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")

  print ('classes:',document_classifier.getClasses())

  nlpPipeline = Pipeline(stages=[
                  documentAssembler, 
                  use,
                  document_classifier
  ])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  clf_pipelineFit = nlpPipeline.fit(empty_data)

  clf_lp_pipeline = LightPipeline(clf_pipelineFit)

  return clf_lp_pipeline


In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_trec50')

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_trec50 download started this may take some time.
Approximate size to download 21.2 MB
[ | ][ / ][ — ][OK!]
classes: [' ENTY_color', ' ENTY_techmeth', ' DESC_manner', ' NUM_volsize', ' ENTY_letter', ' NUM_temp', ' ENTY_body', ' NUM_count', ' ENTY_instru', ' NUM_period', ' NUM_speed', ' DESC_reason', ' ENTY_symbol', ' ENTY_event', ' HUM_desc', ' NUM_perc', ' ENTY_dismed', ' NUM_ord', ' HUM_gr', ' LOC_mount', ' ABBR_abb', ' DESC_desc', ' NUM_dist', ' HUM_title', ' ENTY_lang', ' ENTY_sport', ' ENTY_plant', ' NUM_code', ' NUM_other', ' ENTY_word', ' ENTY_animal', ' ENTY_substance', ' ENTY_veh', ' ENTY_product', ' LOC_state', ' ENTY_religion', ' ENTY_currency', ' NUM_date', ' LOC_country', ' ENTY_cremat', ' NUM_money', ' LOC_other', ' DESC_def', ' LOC_city', ' HUM_ind', ' ENTY_other', ' ENTY_termeq', ' ENTY_food', ' ABBR_exp', ' NUM_weight']

trained on the TREC datasets:

Classify open-domain, fact-based questions into one of the following broad semantic categories: 

```Abbreviation, Description, Entities, Human Beings, Locations or Numeric Values.```

In [0]:
text = 'What was the number of member nations of the U.N. in 2000?'

clf_lp_pipeline.annotate(text)['class']

Out[14]: [' NUM_count']

In [0]:
clf_lp_pipeline.fullAnnotate(text)[0]['class'][0].result

Out[15]: ' NUM_count'

In [0]:
clf_lp_pipeline.fullAnnotate(text)[0]['class'][0].metadata

Out[16]: {' ENTY_dismed': '3.768739E-22', ' ENTY_product': '2.4015744E-24', ' ENTY_techmeth': '1.5787039E-22', ' NUM_speed': '7.9484634E-23', ' NUM_volsize': '2.531511E-25', ' LOC_state': '6.378412E-25', ' NUM_code': '1.454945E-25', ' NUM_count': '0.9992601', ' ENTY_food': '1.3031208E-24', ' ENTY_animal': '1.6743833E-24', ' NUM_period': '6.8075115E-21', ' ENTY_religion': '5.9194734E-23', ' LOC_country': '5.306268E-21', ' LOC_mount': '3.2177816E-25', ' ENTY_termeq': '9.7900855E-26', ' ENTY_color': '1.1446835E-22', ' ENTY_lang': '6.3333907E-24', ' ENTY_sport': '8.0773835E-25', ' DESC_def': '2.4284432E-27', ' HUM_gr': '4.4863106E-21', ' ENTY_symbol': '4.1271923E-25', ' ENTY_currency': '8.156541E-29', ' ENTY_veh': '5.4147004E-22', ' LOC_other': '5.5141072E-11', ' ENTY_word': '5.3265024E-23', ' NUM_temp': '2.0907158E-23', ' NUM_dist': '1.2542656E-24', ' DESC_desc': '1.0926973E-12', ' DESC_manner': '9.258373E-23', ' NUM_ord': '2.2395288E-25', ' NUM_other': '3.977126E-27', ' DESC_reason': '1.1718965E-6', ' NUM_weight': '1.5373857E-24', ' ENTY_instru': '5.9354656E-21', ' ENTY_letter': '1.1453239E-25', ' ENTY_event': '3.706315E-25', ' ENTY_substance': '6.8908435E-25', ' ABBR_exp': '5.6048268E-24', ' ENTY_body': '6.423101E-23', ' ENTY_other': '7.377999E-4', ' NUM_money': '1.6745676E-25', ' LOC_city': '4.7003377E-22', ' NUM_date': '5.212251E-16', ' NUM_perc': '6.3761284E-24', ' ABBR_abb': '7.1010134E-26', ' ENTY_plant': '5.5433757E-24', ' HUM_title': '1.0681952E-24', ' ENTY_cremat': '1.1165376E-24', ' HUM_ind': '8.063819E-7', ' HUM_desc': '4.3701272E-23', 'sentence': '0'}

In [0]:
text = 'What animal was the first mammal successfully cloned from adult cells?'

clf_lp_pipeline.annotate(text)['class']

Out[17]: [' HUM_ind']

In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_cyberbullying')


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_cyberbullying download started this may take some time.
Approximate size to download 21.3 MB
[ | ][ / ][ — ][OK!]
classes: ['sexism', 'neutral', 'racism']

In [0]:
text ='RT @EBeisner @ahall012 I agree with you!! I would rather brush my teeth with sandpaper then watch football with a girl!!'

clf_lp_pipeline.annotate(text)['class']

Out[19]: ['sexism']

In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_fakenews')


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[ | ][OK!]
classes: ['FAKE', 'REAL']

In [0]:
text ='Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton campaign accusation that Donald Trump is a KGB spy is about as weak and baseless a claim as a Salem witch hunt or McCarthy era trial. It’s only because Hillary Clinton is losing that she is lobbing conspiracy theory. Citizen Quasar The way I see it, one of two things will happen: 1. Trump will win by a landslide but the election will be stolen via electronic voting, just like I have been predicting for over a decade, and the American People will accept the skewed election results just like they accept the TSA into their crotches. 2. Somebody will bust a cap in Hillary’s @$$ killing her and the election will be postponed. Follow AMTV!'

clf_lp_pipeline.annotate(text)['class']


Out[21]: ['FAKE']

In [0]:
text ='Sen. Marco Rubio (R-Fla.) is adding a veteran New Hampshire political operative to his team as he continues mulling a possible 2016 presidential bid, the latest sign that he is seriously preparing to launch a campaign later this year.Jim Merrill, who worked for former GOP presidential nominee Mitt Romney and ran his 2008 and 2012 New Hampshire primary campaigns, joined Rubio’s fledgling campaign on Monday, aides to the senator said.Merrill will be joining Rubio’s Reclaim America PAC to focus on Rubio’s New Hampshire and broader Northeast political operations."Marco has always been well received in New Hampshire, and should he run for president, he would be very competitive there," Terry Sullivan, who runs Reclaim America, said in a statement. "Jim certainly knows how to win in New Hampshire and in the Northeast, and will be a great addition to our team at Reclaim America.”News of Merrill’s hire was first reported by The New York Times.'

clf_lp_pipeline.annotate(text)['class']

Out[22]: ['REAL']

In [0]:
sentiment_lp_pipeline = get_clf_lp('sentimentdl_use_twitter', sentiment_dl=True)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ][ / ][OK!]
classes: ['positive', 'negative']

In [0]:
text ='I am SO happy the news came out in time for my birthday this weekend! My inner 7-year-old cannot WAIT!'

sentiment_lp_pipeline.annotate(text)['class']

Out[24]: ['positive']

In [0]:
sentiment_lp_pipeline = get_clf_lp('classifierdl_use_emotion', sentiment_dl=False)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_emotion download started this may take some time.
Approximate size to download 21.3 MB
[ | ][ / ][ — ][ \ ][OK!]
classes: ['joy', 'fear', 'surprise', 'sadness']

In [0]:
sentiment_lp_pipeline.annotate(text)['class']

Out[26]: ['surprise']

## ClassiferDL with Word Embeddings and Text Preprocessing

### Load Dataset

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv
  
dbutils.fs.cp("file:/databricks/driver/news_category_train.csv", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/news_category_test.csv", "dbfs:/")

Out[13]: True

In [0]:
%sh cd /databricks/driver/ && ls -lt 

total 30336
-rw-r--r-- 1 root root 1504408 Apr 23 06:48 news_category_test.csv
-rw-r--r-- 1 root root 24032125 Apr 23 06:48 news_category_train.csv
-rw-r--r-- 1 root root 5502589 Apr 23 06:47 spam_ham_dataset.csv
drwxr-xr-x 2 root root 4096 Apr 23 06:45 ganglia
drwxr-xr-x 2 root root 4096 Apr 23 06:18 logs
drwxr-xr-x 3 root root 4096 Apr 23 06:18 eventlogs
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf

In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/news_category_train.csv")

trainDataset.show(truncate=50)

+--------+--------------------------------------------------+
category| description|
+--------+--------------------------------------------------+
Business| Short sellers, Wall Street's dwindling band of...|
Business| Private investment firm Carlyle Group, which h...|
Business| Soaring crude prices plus worries about the ec...|
Business| Authorities have halted oil export flows from ...|
Business| Tearaway world oil prices, toppling records an...|
Business| Stocks ended slightly higher on Friday but sta...|
Business| Assets of the nation's retail money market mut...|
Business| Retail sales bounced back a bit in July, and n...|
Business|" After earning a PH.D. in Sociology, Danny Baz...|
Business| Short sellers, Wall Street's dwindling band o...|
Business| Soaring crude prices plus worries about the e...|
Business| OPEC can do nothing to douse scorching oil pr...|
Business| Non OPEC oil exporters should consider increa...|
Business| WASHINGTON/NEW YORK (Reuters) - The auction fo...|
Business| The dollar tumbled broadly on Friday after da...|
Business|If you think you may need to help your elderly ...|
Business|The purchasing power of kids is a big part of w...|
Business|There is little cause for celebration in the st...|
Business|The US trade deficit has exploded 19 to a recor...|
Business|Oil giant Shell could be bracing itself for a t...|
+--------+--------------------------------------------------+
only showing top 20 rows

In [0]:
trainDataset.count()


Out[16]: 120000

In [0]:
from pyspark.sql.functions import col

trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
category|count|
+--------+-----+
 World|30000|
Sci/Tech|30000|
 Sports|30000|
Business|30000|
+--------+-----+

In [0]:
testDataset = spark.read \
      .option("header", True) \
      .csv("/news_category_test.csv")

testDataset.show(truncate=50)

+--------+--------------------------------------------------+
category| description|
+--------+--------------------------------------------------+
Business|Unions representing workers at Turner Newall ...|
Sci/Tech| TORONTO, Canada A second team of rocketeers...|
Sci/Tech| A company founded by a chemistry researcher at...|
Sci/Tech| It's barely dawn when Mike Fitzpatrick starts ...|
Sci/Tech| Southern California's smog fighting agency wen...|
Sci/Tech|"The British Department for Education and Skill...|
Sci/Tech|"confessed author of the Netsky and Sasser viru...|
Sci/Tech|\\FOAF/LOAF and bloom filters have a lot of in...|
Sci/Tech|"Wiltshire Police warns about ""phishing"" afte...|
Sci/Tech|In its first two years, the UK's dedicated card...|
Sci/Tech| A group of technology companies including Tex...|
Sci/Tech| Apple Computer Inc.&lt;AAPL.O&gt; on Tuesday ...|
Sci/Tech| Free Record Shop, a Dutch music retail chain,...|
Sci/Tech|A giant 100km colony of ants which has been di...|
Sci/Tech| "Dolphin groups, or ""pods""|
Sci/Tech|Tyrannosaurus rex achieved its massive size due...|
Sci/Tech| Scientists have discovered irregular lumps be...|
Sci/Tech| ESAs Mars Express has relayed pictures from o...|
Sci/Tech|When did life begin? One evidential clue stems ...|
Sci/Tech|update Earnings per share rise compared with a ...|
+--------+--------------------------------------------------+
only showing top 20 rows

In [0]:
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
category|count|
+--------+-----+
 World| 1900|
Sci/Tech| 1900|
 Sports| 1900|
Business| 1900|
+--------+-----+

In [0]:
# if we want to split the dataset
'''(trainData, testData) = trainDataset.randomSplit([0.7, 0.3], seed = 100)
print("Train Dataset Count: " + str(trainData.count()))
print("Test Dataset Count: " + str(testData.count()))'''


Out[34]: '(trainData, testData) = trainDataset.randomSplit([0.7, 0.3], seed = 100)\nprint("Train Dataset Count: " + str(trainData.count()))\nprint("Test Dataset Count: " + str(testData.count()))'

In [0]:
document_assembler = DocumentAssembler() \
      .setInputCol("description") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("lemma")


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][OK!]

### with Glove 100d embeddings

In [0]:

glove_embeddings = WordEmbeddingsModel.pretrained("glove_100d") \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("category")\
      .setMaxEpochs(10)\
      .setEnableOutputLogs(True)
      #.setOutputLogsPath('logs')

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
'''
default classifierDL params:

maxEpochs -> 10,
lr -> 5e-3f,
dropout -> 0.5f,
batchSize -> 64,
enableOutputLogs -> false,
verbose -> Verbose.Silent.id,
validationSplit -> 0.0f,
outputLogsPath -> ""
'''

Out[58]: '\ndefault classifierDL params:\n\nmaxEpochs -> 10,\nlr -> 5e-3f,\ndropout -> 0.5f,\nbatchSize -> 64,\nenableOutputLogs -> false,\nverbose -> Verbose.Silent.id,\nvalidationSplit -> 0.0f,\noutputLogsPath -> ""\n'

In [0]:
# Train (8 min for 10 epochs)

clf_pipelineModel = clf_pipeline.fit(trainDataset)

In [0]:
# due to bug in cluster mode (https://github.com/JohnSnowLabs/spark-nlp/issues/857) , as a workaround, you can just save the fitted model and then load back from dbfs and then transform on the test set. 
clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/ClassifierDL_wordemb_g100d')
classsifierdlmodel_loaded = ClassifierDLModel.load('dbfs:/databricks/driver/models/ClassifierDL_wordemb_g100d')


clf_pipeline_pred = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            clf_pipelineModel.stages[-2],
            classsifierdlmodel_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

result = clf_pipeline_pred.fit(empty_data).transform(testDataset)

In [0]:
preds_df = result.select('category','description',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))


precision recall f1-score support

 Business 0.84 0.83 0.84 1926
 Sci/Tech 0.88 0.82 0.85 2020
 Sports 0.97 0.95 0.96 1944
 World 0.84 0.94 0.89 1710

 accuracy 0.88 7600
 macro avg 0.88 0.88 0.88 7600
weighted avg 0.88 0.88 0.88 7600

## Getting prediction from Trained model

In [0]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(clf_pipelineModel)

In [0]:
text='''
Fearing the fate of Italy, the centre-right government has threatened to be merciless with those who flout tough restrictions. 
As of Wednesday it will also include all shops being closed across Greece, with the exception of supermarkets. Banks, pharmacies, pet-stores, mobile phone stores, opticians, bakers, mini-markets, couriers and food delivery outlets are among the few that will also be allowed to remain open.
'''
result = light_model.annotate(text)

result['class']

Out[43]: ['Sci/Tech']

### with Bert Embeddings

In [0]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")
    

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][OK!]

In [0]:
bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
     .setInputCols(["document",'token'])\
     .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("category")\
      .setMaxEpochs(3)\
      .setLr(0.001)\
      .setBatchSize(8)\
      .setEnableOutputLogs(True)
      #.setOutputLogsPath('logs')

bert_clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            classsifierdl])

small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[ | ][ / ][ — ][OK!]

In [0]:
# training will take some time due to Bert (use GPU runtime when possible)

bert_clf_pipelineModel = bert_clf_pipeline.fit(trainDataset)

In [0]:
bert_clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/small_bert')
bert_clf_loaded = ClassifierDLModel.load('dbfs:/databricks/driver/models/small_bert')


bert_clf_pipeline_pred = Pipeline(
    stages=[document_assembler, 
            tokenizer, 
            bert_embeddings,
            bert_clf_pipelineModel.stages[-2],
            bert_clf_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

result = bert_clf_pipeline_pred.fit(empty_data).transform(testDataset)

In [0]:
preds_df = result.select('category','description',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))

precision recall f1-score support

 Business 0.83 0.82 0.82 1916
 Sci/Tech 0.86 0.82 0.84 1982
 Sports 0.96 0.93 0.94 1956
 World 0.84 0.92 0.88 1746

 accuracy 0.87 7600
 macro avg 0.87 0.87 0.87 7600
weighted avg 0.87 0.87 0.87 7600

## ClassifierDL with Universal Sentence Embeddings

In [0]:
# actual content is inside description column
document = DocumentAssembler()\
      .setInputCol("description")\
      .setOutputCol("document")
    
# we can also use sentece detector here if we want to train on and get predictions for each sentence

use = UniversalSentenceEncoder.pretrained()\
     .setInputCols(["document"])\
     .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("category")\
      .setMaxEpochs(5)\
      .setLr(0.001)\
      .setBatchSize(8)\
      .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
use_pipelineModel = use_clf_pipeline.fit(trainDataset)
# 5 epochs takes around 10 min

In [0]:
%sh cd ~/annotator_logs && ls -l


total 8
-rw-r--r-- 1 root root 320 Apr 23 07:18 ClassifierDLApproach_0033b893b924.log
-rw-r--r-- 1 root root 458 Apr 23 07:34 ClassifierDLApproach_f592fc5e19a5.log

In [0]:
%sh cat ~/annotator_logs/ClassifierDLApproach_0033b893b924.log

Training started - epochs: 3 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/3 - 84.48s - loss: 13299.423 - acc: 0.85519165 - batches: 15000
Epoch 1/3 - 83.01s - loss: 13093.5205 - acc: 0.86995 - batches: 15000
Epoch 2/3 - 83.64s - loss: 12993.676 - acc: 0.87695 - batches: 15000

### Saving & loading back the trained model

In [0]:
use_pipelineModel.stages

Out[27]: [DocumentAssembler_bca6db72dc40,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 ClassifierDLModel_670f5c3b6729]

In [0]:
use_pipelineModel.stages[2].write().overwrite().save('/databricks/driver/models/ClassifierDL_USE_20200211_e5')

In [0]:
classsifierdlmodel = ClassifierDLModel.load('dbfs:/databricks/driver/models/ClassifierDL_USE_20200211_e5')

In [0]:
clf_lp = get_clf_lp('dbfs:/databricks/driver/models/ClassifierDL_USE_20200211_e5', sentiment_dl=False,  pretrained=False)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classes: ['World', 'Sci/Tech', 'Sports', 'Business']

In [0]:
clf_lp.annotate(text)['class']

Out[33]: ['World']

# SentimentDL Classifier

see also here >> `https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/SentimentDL_train_multiclass_sentiment_classifier.ipynb`

In [0]:

!wget -q aclimdb_train.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_train.csv
!wget -q aclimdb_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_test.csv
  
dbutils.fs.cp("file:/databricks/driver/aclimdb_train.csv", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/aclimdb_test.csv", "dbfs:/")

Out[34]: True

In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/aclimdb_train.csv")

trainDataset.show()

+--------------------+--------+
 text| label|
+--------------------+--------+
This is an Excell...|positive|
The Sarah Silverm...|positive|
"Prom Night" is a...|negative|
So often a band w...|positive|
"Pet Sematary" is...|positive|
I watched the fil...|negative|
Boy this movie ha...|negative|
Checking the spoi...|negative|
Despite its rathe...|positive|
Absolute masterpi...|positive|
The tweedy profes...|positive|
A movie best summ...|negative|
Take young, prett...|negative|
For months I've b...|negative|
"Batman: The Myst...|positive|
Well, it was funn...|negative|
I have seen the s...|positive|
Brainless film ab...|negative|
Leave it to geniu...|negative|
Seven Pounds star...|positive|
+--------------------+--------+
only showing top 20 rows

In [0]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
   .setInputCols(["document"])\
   .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(stages = [
    document,
    use,
    sentimentdl
    ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
pipelineModel = pipeline.fit(trainDataset)

# MultiLabel Classifier

see also here >> `https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/MultiClassifierDL_train_multi_label_toxic_classifier.ipynb`

In [0]:
!curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_train.snappy.parquet'
!curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_test.snappy.parquet'

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0 63 2702k 63 1714k 0 0 1786k 0 0:00:01 --:--:-- 0:00:01 1784k100 2702k 100 2702k 0 0 2726k 0 --:--:-- --:--:-- --:--:-- 2724k
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0100 289k 100 289k 0 0 424k 0 --:--:-- --:--:-- --:--:-- 423k

In [0]:
trainDataset = spark.read.parquet("/toxic_train.snappy.parquet").repartition(120)
testDataset = spark.read.parquet("/toxic_test.snappy.parquet").repartition(10)

In [0]:
# Let's use shrink to remove new lines in the comments
document = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")\
  .setCleanupMode("shrink")

# Here we use the state-of-the-art Universal Sentence Encoder model from TF Hub
embeddings = UniversalSentenceEncoder.pretrained() \
  .setInputCols(["document"])\
  .setOutputCol("sentence_embeddings")

# We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes
# We will use only 5 Epochs but feel free to increase it on your own dataset
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("category")\
  .setLabelColumn("labels")\
  .setBatchSize(128)\
  .setMaxEpochs(5)\
  .setLr(1e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setValidationSplit(0.1)

pipeline = Pipeline(
    stages = [
        document,
        embeddings,
        multiClassifier
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
pipelineModel = pipeline.fit(trainDataset)

In [0]:
print(pipeline.fit(testDataset).stages[2].getClasses())
# >> ['toxic', 'severe_toxic', 'identity_hate', 'insult', 'obscene', 'threat']

None

In [0]:
# pred output: [toxic, obscene]

# Case Study: Conference Title Classification

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/title_conference.csv
  
dbutils.fs.cp("file:/databricks/driver/title_conference.csv", "dbfs:/")

Out[49]: True

In [0]:
import pandas as pd
df = pd.read_csv('title_conference.csv')
df

Out[50]:

,Title,Conference
0,Innovation in Database Management: Computer Science vs. Engineering.,VLDB
1,High performance prime field multiplication for GPU.,ISCAS
2,enchanted scissors: a scissor interface for support in cutting and interactive fabrication.,SIGGRAPH
3,Detection of channel degradation attack by Intermediary Node in Linear Networks.,INFOCOM
4,Pinning a Complex Network through the Betweenness Centrality Strategy.,ISCAS
...,...,...
2502,A new QR-decomposition based recursive frequency estimator for multiple sinusoids in impulsive n...,ISCAS
2503,CNN Implementation of Spin Filters for Electronic Speckle Pattern Interferometry Applications.,ISCAS
2504,FaceKit: A Database Interface Design Toolkit.,VLDB
2505,On the trade-off between the number of scrolls and the operating frequency of the chaotic attrac...,ISCAS


In [0]:
df.Conference.value_counts()

Out[51]: ISCAS 864
INFOCOM 515
VLDB 423
WWW 379
SIGGRAPH 326
Name: Conference, dtype: int64

In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/title_conference.csv")

(trainingData, testData) = trainDataset.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))


Training Dataset Count: 2030
Test Dataset Count: 477

In [0]:
document = DocumentAssembler()\
      .setInputCol("Title")\
      .setOutputCol("document")
    
# we can also use sentece detector here if we want to train on and get predictions for each sentence

use = UniversalSentenceEncoder.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("Conference")\
      .setMaxEpochs(20)\
      .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
use_pipelineModel = use_clf_pipeline.fit(trainingData)

# 20 epochs takes around 22 seconds !

In [0]:
use_pipelineModel.stages

Out[55]: [DocumentAssembler_2ab5c09c141d,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 ClassifierDLModel_2e86df876703]

In [0]:
use_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/use_clf')
use_clf_loaded = ClassifierDLModel.load('dbfs:/databricks/driver/models/use_clf')

use_clf_pipeline_pred = Pipeline(
    stages=[document, 
            use_pipelineModel.stages[-2],
            use_clf_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

result = use_clf_pipeline_pred.fit(empty_data).transform(testData)

In [0]:
result.select('Title','Conference',"class.result").show(10, truncate=80)


+--------------------------------------------------------------------------------+----------+----------+
 Title|Conference| result|
+--------------------------------------------------------------------------------+----------+----------+
 "From ""Think Like a Vertex"" to ""Think Like a Graph""."| VLDB|[SIGGRAPH]|
 1, 000 Tables Inside the From.| VLDB| [VLDB]|
 10Gbit/s 2mW inductorless transimpedance amplifier.| ISCAS| [ISCAS]|
3D reconstruction from planar points: A candidate method for authentication o...| ISCAS| [ISCAS]|
 7-decades tunable translinear SiGe BiCMOS 3-phase sinusoidal oscillator.| ISCAS| [ISCAS]|
 A 0.18-&micro;m CMOS Squarer Circuit for a Non-Coherent UWB Receiver.| ISCAS| [ISCAS]|
 A 10-Bit 200-MS/s digitally-calibrated pipelined ADC using switching opamps.| ISCAS| [ISCAS]|
 A 10-bit 2GHz Current-Steering CMOS D/A Converter.| ISCAS| [ISCAS]|
 A 13-bit, low-power, compact ADC suitable for sensor applications.| ISCAS| [ISCAS]|
A 14-b 32MS/s Pipelined ADC with Novel Fast-convergence Comprehensive Backgro...| ISCAS| [ISCAS]|
+--------------------------------------------------------------------------------+----------+----------+
only showing top 10 rows

In [0]:
# We are going to use sklearn to evalute the results on test dataset
preds_df = result.select('Conference','Title',"class.result").toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['Conference']))


precision recall f1-score support

 INFOCOM 0.71 0.80 0.75 88
 ISCAS 0.87 0.83 0.85 186
 SIGGRAPH 0.63 0.67 0.65 48
 VLDB 0.68 0.60 0.64 78
 WWW 0.63 0.68 0.65 77

 accuracy 0.74 477
 macro avg 0.71 0.71 0.71 477
weighted avg 0.75 0.74 0.74 477

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

preds = pd.DataFrame(confusion_matrix(preds_df['result'], preds_df['Conference']), columns = np.unique(preds_df['Conference']), index =  np.unique(preds_df['Conference']))
preds

Out[59]:

,INFOCOM,ISCAS,SIGGRAPH,VLDB,WWW
INFOCOM,70,8,1,2,7
ISCAS,13,154,11,5,3
SIGGRAPH,1,6,32,4,5
VLDB,6,8,2,47,15
WWW,8,1,5,11,52


End of Notebook # 5